# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor

In [3]:
features = ['Was_home',
 'Rating_difference',
 'Min_2',
 'Gls_2',
 'Ast_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'FPL_points_2',
 'Min_4',
 'Gls_4',
 'Ast_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'FPL_points_4',
 'Min_30',
 'Gls_30',
 'Ast_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'FPL_points_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

35

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

In [6]:
# df = df[df["GW"] == 2]

In [7]:
# X_test = df[df["Season"] == CURRENT_SEASON][features]
X_test = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [8]:
# X_test[X_test["FPL_pos"] == "MID"]["Gls_4"].unique()

In [9]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'DEF', 'FPL_points',
       'FPL_points_2', 'FPL_points_30', 'FPL_points_4', 'FPL_pos', 'FWD', 'GW',
       'Gls_2', 'Gls_30', 'Gls_4', 'MID', 'Min_2', 'Min_30', 'Min_4',
       'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Team',
       'Team_CS_2', 'Team_CS_30', 'Team_CS_4', 'Team_rating', 'Team_result_2',
       'Team_result_30', 'Team_result_4', 'Team_score_2', 'Team_score_30',
       'Team_score_4', 'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30',
       'xG_4', 'xP', 'Finished', 'Date', 'Price'],
      dtype='object')

In [10]:
class PositionalModel:
    def __init__(self,  model_GK=None, model_DEF=None, model_MID=None, model_FWD=None):
        if model_GK:
            self.model_GK = model_GK
        else:
            self.model_GK = XGBRegressor(random_state=42, verbosity = 0,
            # n_estimators=500,
            # early_stopping_rounds=5,
            # learning_rate=0.2
            )
        
        if model_DEF:
            self.model_DEF = model_DEF
        else:
            self.model_DEF = XGBRegressor(random_state=42, verbosity = 0,
            n_estimators=500,
            early_stopping_rounds=5,
            learning_rate=0.2
            )
        
        if model_MID:
            self.model_MID = model_MID
        else:
            self.model_MID = XGBRegressor(random_state=42, verbosity = 0,
            n_estimators=500,
            early_stopping_rounds=5,
            learning_rate=0.2
            )
        
        if model_FWD:
            self.model_FWD = model_FWD
        else:
            self.model_FWD = XGBRegressor(random_state=42, verbosity = 0,
            n_estimators=500,
            early_stopping_rounds=5,
            learning_rate=0.2
            )
        
        
    def fit(self, X, y, X_valid, y_valid):
        # print(y[y["FPL_pos"] == "GK"].shape)
        self.model_GK.fit(X[X["FPL_pos"] == "GK"][features], y[y["FPL_pos"] == "GK"][to_predict],
        # eval_set=[(X_valid[X_valid["FPL_pos"] == "GK"][features], y_valid[y_valid["FPL_pos"] == "GK"][to_predict])],
        verbose=False)
        self.model_DEF.fit(X[X["FPL_pos"] == "DEF"][features], y[y["FPL_pos"] == "DEF"][to_predict],
        eval_set=[(X_valid[X_valid["FPL_pos"] == "DEF"][features], y_valid[y_valid["FPL_pos"] == "DEF"][to_predict])],
        verbose=False)
        self.model_MID.fit(X[X["FPL_pos"] == "MID"][features], y[y["FPL_pos"] == "MID"][to_predict],
        eval_set=[(X_valid[X_valid["FPL_pos"] == "MID"][features], y_valid[y_valid["FPL_pos"] == "MID"][to_predict])],
        verbose=False)
        self.model_FWD.fit(X[X["FPL_pos"] == "FWD"][features], y[y["FPL_pos"] == "FWD"][to_predict],
        eval_set=[(X_valid[X_valid["FPL_pos"] == "FWD"][features], y_valid[y_valid["FPL_pos"] == "FWD"][to_predict])],
        verbose=False)
        
    def predict(self, X):
        output = []
        i = 0
        for ind, row in X.iterrows():
            # print(i, row["Name_original"], row["FPL_points_4"], X.head(i+1).tail(1)["FPL_points_4"])
            if row["FPL_pos"] == "GK":
                # print(row["FPL_pos"])
                output.append( self.model_GK.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "DEF":
                # print(row["FPL_pos"])
                output.append( self.model_DEF.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "MID":
                # print(row["FPL_pos"])
                output.append( self.model_MID.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "FWD":
                # print(row["FPL_pos"])
                output.append( self.model_FWD.predict( X.head(i+1).tail(1)[features] ) )
            i += 1
        return np.array(output).flatten()

In [11]:
file1 = open("models/GBR_GK.pkl",'rb')
file2 = open("models/GBR_DEF.pkl",'rb')
file3 = open("models/GBR_MID.pkl",'rb')
file4 = open("models/GBR_FWD.pkl",'rb')

model_GK = pickle.load(file1)
model_DEF = pickle.load(file2)
model_MID = pickle.load(file3)
model_FWD = pickle.load(file4)

model = PositionalModel(model_GK, model_DEF, model_MID, model_FWD)

In [12]:
model

In [13]:
model_MID.predict( X_test.head(10)[features] )

array([1.1904162, 1.1904162, 1.1904162, 0.6569128, 0.6569128, 1.1904162,
       1.1904162, 0.6569128, 0.6569128, 1.1904162], dtype=float32)

In [14]:
model_MID.predict( X_test.head(12)[features] )

array([1.1904162, 1.1904162, 1.1904162, 0.6569128, 0.6569128, 1.1904162,
       1.1904162, 0.6569128, 0.6569128, 1.1904162, 1.1904162, 1.1904162],
      dtype=float32)

In [15]:
model.predict( X_test.head(5) )

array([4.588393  , 3.9490452 , 4.588393  , 0.76540446, 0.5811391 ],
      dtype=float32)

In [16]:
model.predict( X_test.head(3) )

array([4.588393 , 3.9490452, 4.588393 ], dtype=float32)

In [17]:
model.predict( X_test.head(4) )

array([4.588393  , 3.9490452 , 4.588393  , 0.76540446], dtype=float32)

In [18]:
X_test.head(5)

,Unnamed: 0,Ast_2,Ast_30,Ast_4,DEF,FPL_points,FPL_points_2,FPL_points_30,FPL_points_4,FPL_pos,...,xA_2,xA_30,xA_4,xG_2,xG_30,xG_4,xP,Finished,Date,Price
15408,1691,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-12,45.0
15409,1669,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-19,45.0
15410,1692,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-26,45.0
15411,1693,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-09-02,45.0
15412,1670,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-09-16,45.0


In [19]:
X_test.head(3)

,Unnamed: 0,Ast_2,Ast_30,Ast_4,DEF,FPL_points,FPL_points_2,FPL_points_30,FPL_points_4,FPL_pos,...,xA_2,xA_30,xA_4,xG_2,xG_30,xG_4,xP,Finished,Date,Price
15408,1691,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-12,45.0
15409,1669,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-19,45.0
15410,1692,0.0,0.0,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,0.05,0.0125,0.025,0.0,0.0375,0.0,NaN,False,2023-08-26,45.0


In [20]:
X_test.head(3)[features]

,Was_home,Rating_difference,Min_2,Gls_2,Ast_2,xG_2,xA_2,Team_CS_2,Team_score_2,Opp_score_2,...,xG_30,xA_30,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,FPL_points_30,DEF,FWD,MID
15408,1.0,221.646484,18.0,0.0,0.0,0.0,0.05,0.0,1.0,2.5,...,0.0375,0.0125,0.375,0.875,1.125,1.25,0.428571,0.0,1.0,0.0
15409,0.0,116.732910,18.0,0.0,0.0,0.0,0.05,0.0,1.0,2.5,...,0.0375,0.0125,0.375,0.875,1.125,1.25,0.428571,0.0,1.0,0.0
15410,1.0,54.812988,18.0,0.0,0.0,0.0,0.05,0.0,1.0,2.5,...,0.0375,0.0125,0.375,0.875,1.125,1.25,0.428571,0.0,1.0,0.0


# Predicting

In [21]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [22]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [23]:
X_test

,Unnamed: 0,Ast_2,Ast_30,Ast_4,DEF,FPL_points,FPL_points_2,FPL_points_30,FPL_points_4,FPL_pos,...,xA_2,xA_30,xA_4,xG_2,xG_30,xG_4,xP,Finished,Date,Price
15408,1691,0.0,0.000000,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,5.000000e-02,0.0125,2.500000e-02,0.000000e+00,0.037500,0.000000e+00,NaN,False,2023-08-12,45.0
15409,1669,0.0,0.000000,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,5.000000e-02,0.0125,2.500000e-02,0.000000e+00,0.037500,0.000000e+00,NaN,False,2023-08-19,45.0
15410,1692,0.0,0.000000,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,5.000000e-02,0.0125,2.500000e-02,0.000000e+00,0.037500,0.000000e+00,NaN,False,2023-08-26,45.0
15411,1693,0.0,0.000000,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,5.000000e-02,0.0125,2.500000e-02,0.000000e+00,0.037500,0.000000e+00,NaN,False,2023-09-02,45.0
15412,1670,0.0,0.000000,0.0,0.0,NaN,0.5,0.428571,0.333333,FWD,...,5.000000e-02,0.0125,2.500000e-02,0.000000e+00,0.037500,0.000000e+00,NaN,False,2023-09-16,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28920,6132,0.0,0.033333,0.0,0.0,NaN,1.0,1.733333,0.750000,MID,...,2.137179e-15,0.0200,4.718448e-16,1.554312e-15,0.006667,6.383782e-16,NaN,False,2024-04-20,50.0
28921,6133,0.0,0.033333,0.0,0.0,NaN,1.0,1.733333,0.750000,MID,...,2.137179e-15,0.0200,4.718448e-16,1.554312e-15,0.006667,6.383782e-16,NaN,False,2024-04-27,50.0
28922,6126,0.0,0.033333,0.0,0.0,NaN,1.0,1.733333,0.750000,MID,...,2.137179e-15,0.0200,4.718448e-16,1.554312e-15,0.006667,6.383782e-16,NaN,False,2024-05-04,50.0
28923,6134,0.0,0.033333,0.0,0.0,NaN,1.0,1.733333,0.750000,MID,...,2.137179e-15,0.0200,4.718448e-16,1.554312e-15,0.006667,6.383782e-16,NaN,False,2024-05-11,50.0


In [24]:
%%time
preds = get_predictions(model, df, X_test)

13517 (13517, 49)
CPU times: total: 2min 42s
Wall time: 52.5 s


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [25]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [26]:
# next gameweek
preds[preds["GW"] == 3].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1708,Bryan Mbeumo,3.0,2023-24,Brentford,Crystal Palace,1.0,1822.547119,1763.994995,FWD,65.0,7.648046
2880,Danny Welbeck,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,FWD,60.0,7.524686
875,Anthony Martial,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,FWD,65.0,7.278353
4284,Folarin Balogun,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,FWD,45.0,7.170025
7086,Kai Havertz,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,FWD,75.0,6.835954
7011,Julio Enciso,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,FWD,55.0,6.673560
3222,Diogo Teixeira da Silva,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,FWD,80.0,6.465848
7049,Julián Álvarez,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,65.0,6.363253
3526,Eddie Nketiah,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,FWD,55.0,6.289495
4056,Evan Ferguson,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,FWD,60.0,5.939478


In [27]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
9323,Michail Antonio,10.0,2023-24,West Ham,Everton,1.0,1777.551270,1699.505859,FWD,60.0,8.841724
9325,Michail Antonio,12.0,2023-24,West Ham,Nott'ham Forest,1.0,1777.551270,1671.868774,FWD,60.0,8.841724
9346,Michail Antonio,33.0,2023-24,West Ham,Fulham,1.0,1777.551270,1745.365723,FWD,60.0,8.841724
9320,Michail Antonio,7.0,2023-24,West Ham,Sheffield United,1.0,1777.551270,1637.473755,FWD,60.0,8.841724
9335,Michail Antonio,22.0,2023-24,West Ham,Bournemouth,1.0,1777.551270,1662.536621,FWD,60.0,8.841724
9327,Michail Antonio,14.0,2023-24,West Ham,Crystal Palace,1.0,1777.551270,1763.994995,FWD,60.0,8.841724
9330,Michail Antonio,17.0,2023-24,West Ham,Wolves,1.0,1777.551270,1715.631348,FWD,60.0,8.841724
9350,Michail Antonio,37.0,2023-24,West Ham,Luton,1.0,1777.551270,1602.327637,FWD,60.0,8.841724
9341,Michail Antonio,28.0,2023-24,West Ham,Burnley,1.0,1777.551270,1722.802979,FWD,60.0,8.841724
12267,Taiwo Awoniyi,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1671.868774,1637.473755,FWD,65.0,7.878557


In [28]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Diogo Teixeira da Silva,258.620880
Julián Álvarez,256.045471
Folarin Balogun,229.656647
Kai Havertz,221.007843
Anthony Martial,217.455536
Danny Welbeck,210.814545
Eddie Nketiah,204.299179
Alexander Isak,202.970825
Bryan Mbeumo,200.288620


# Saving to file

In [29]:
preds.to_csv("predictions/preds_next_season.csv", index=False)